# Prompt Engineering



In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
import getpass

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:\n")

GROQ API Key:
··········


In [3]:
from groq import Groq

client = Groq(
    api_key= os.environ.get("GROQ_API_KEY")
)

#### helper function

This helper function will make it easier to use prompts and look at the generated outputs:

In [15]:
def get_completion(prompt, model="gemma2-9b-it", **kwargs):  # llama3-70b-8192
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Prompting guidelines




### Write clear and specific instructions



#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [8]:
text = f"""
Machine learning is a field of study that focuses \
on developing algorithms and models that allow \
computers to learn from and make predictions or \
decisions based on data. It involves training \
a model on a dataset, evaluating its performance, \
and using it to make predictions on new, unseen data.
"""

prompt = f"""
Summarize the given text into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

Machine learning is the field of study dedicated to creating algorithms that enable computers to learn from data and make predictions. 





#### Tactic 2: Ask for a structured output
- JSON, HTML

In [14]:
prompt = f"""
Generate list of three top Indian Business Schools \
with its City.
Provide them in JSON format.

"""
response = get_completion(prompt)
print(response)

```json
[
  { "rank": "1", "school": "Indian Institute of Management Ahmedabad (IIMA)", "city": "Ahmedabad" },
  { "rank": "2", "school": "Indian Institute of Management Bangalore (IIMB)", "city":  "Bangalore" },
  { "rank": "3", "school": "Indian Institute of Management Calcutta (IIMC)", "city": "Kolkata" }
]
```  



**Please Note:** It is open to debate whether any list categorically designates just three as geographically limited "Top Institutions." Prestige can be subject to changing parameters. 



#### Tactic 3: Ask the model to check whether conditions are satisfied

In [ ]:
cooking_instruction = """
To prepare a quick and delicious plate of Maggi noodles, \
start by boiling 1.5 to 2 cups of water. Once the water is boiling, break the \
Maggi noodle cake into smaller pieces and add them to the boiling water.\
Let the noodles cook for about 2 minutes, stirring occasionally. Open the \
flavoring sachet and add its contents to the noodles, mixing well. Cook for another 1-2 minutes \
until the noodles reach your desired doneness. Remove from heat, let it cool for a minute,\
and transfer to a bowl or plate. Customize your Maggi noodles by adding vegetables, \
eggs, or protein if desired. Enjoy your hot and tasty Maggi noodles!
"""

prompt = f"""
You will be provided with a set of instructions.
If it contains a sequence of instructions,
rewrite the instructions in the following format:

Step 1: ...
Step 2: ...
etc..

Rewrite the following instructions accordingly:

\"\"\"{cooking_instruction}\"\"\"
"""
response = get_completion(prompt)
print("Processed Text: ")
print(response)

Processed Text: 
Here are the rewritten instructions:

Step 1: Boil 1.5 to 2 cups of water.

Step 2: Break the Maggi noodle cake into smaller pieces and add them to the boiling water.

Step 3: Let the noodles cook for about 2 minutes, stirring occasionally.

Step 4: Open the flavoring sachet and add its contents to the noodles, mixing well.

Step 5: Cook for another 1-2 minutes until the noodles reach your desired doneness.

Step 6: Remove from heat, let it cool for a minute, and transfer to a bowl or plate.

Step 7: Customize your Maggi noodles by adding vegetables, eggs, or protein if desired.

Step 8: Enjoy your hot and tasty Maggi noodles!


In [ ]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \
walk in the park. The flowers are blooming, and the \
trees are swaying gently in the breeze. People \
are out and about, enjoying the lovely weather. \
Some are having picnics, while others are playing \
games or simply relaxing on the grass. It's a \
perfect day to spend time outdoors and appreciate the \
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes.
If it contains a sequence of instructions, \
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [ ]:
prompt = f"""
Sample1: The Pt has a pain-scale is 6/10. Classify the patiet as Uncomplicated or Complicated ? Answer: Uncomplicated
Sample2: The Pt has a pain-scale is 7/10. Classify the patiet as Uncomplicated or Complicated ? Answer: Complicated
Sample3: The Pt pain-scale is at 3/10. Is he Uncomplicated or Complicated ? Answer:
"""
response = get_completion(prompt)
print(response)

Based on the pattern, I would classify the patient as Uncomplicated.

The reasoning is that a pain scale of 6/10 or below is considered Uncomplicated, while a pain scale of 7/10 or above is considered Complicated.


### Descriptive & Contextual (more time to think)



#### Tactic 1: Specify the steps required to complete a task

In [ ]:
text = f"""
King Ashoka, also known as Ashoka the Great, was an \
Indian emperor who ruled the Maurya Dynasty from \
c. 268 to 232 BCE. He is widely regarded as one of \
India's greatest rulers. After a series of violent \
military campaigns, Ashoka embraced Buddhism and \
adopted a policy of non-violence and compassion. \
He promoted the spread of Buddhism throughout his \
empire and erected numerous pillars and edicts \
inscribed with Buddhist teachings. Ashoka's rule is \
notable for his efforts towards social welfare, \
including the establishment of hospitals, veterinary \
clinics, and the protection of wildlife. His reign is \
considered a golden age of peace, prosperity, and \
religious harmony in ancient India.
"""

# example 1
prompt_1 = f"""
Perform the following actions:
1 - Summarize the following text in 1 sentence.
2 - Translate the summary into Hindi.
3 - List each name in the Hindi summary.
4 - Output a JSON object that contains the qualities of Ashoka.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the answers:

**1. Summary in 1 sentence:**
King Ashoka, a renowned Indian emperor, ruled the Maurya Dynasty from 268 to 232 BCE, promoting Buddhism, non-violence, and social welfare after a violent past.

**2. Hindi Translation:**
राजा अशोक, एक प्रसिद्ध भारतीय सम्राट, 268 से 232 ईसा पूर्व मौर्य वंश का शासन किया, हिंसा के बाद बौद्ध धर्म, अहिंसा और सामाजिक कल्याण को बढ़ावा दिया।

**3. List of names in Hindi summary:**
* अशोक (Ashok)

**4. JSON object with qualities of Ashoka:**
```
{
  "qualities": [
    "renowned",
    "promoter of Buddhism",
    "non-violent",
    "compassionate",
    "social welfare advocate",
    "peaceful",
    "prosperous",
    "religious harmonizer"
  ]
}
```


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [ ]:
prompt = """
Evaluate the accuracy of the student's mathematical derivation.

Question:
A group of friends decides to embark on a road trip adventure.
They plan to rent a car and share the expenses equally.
The car rental company offers a daily rental rate of $50
and an additional charge of $0.25 per mile driven.
Each friend contributes an equal amount towards the total expenses.
If the group drives a total of 'm' miles during the trip,
what is the cost per person as a function of 'm'?

Student's Solution:
Let 'm' represent the total miles driven.
Cost Calculation:
1. Daily rental cost: $50
2. Mileage cost: $0.25 * m
Total cost: $50 + ($0.25 + m)
Cost per person: ($50 + ($0.25 + m)) / number_of_friends

Evaluate the mathematical derivation provided by the student and determine
if it accurately calculates the cost per person as a
function of the total miles driven.
"""
response = get_completion(prompt)
print(response)

Let's break down the student's solution and evaluate its accuracy.

**Cost Calculation:**

1. Daily rental cost: $50 (Correct)
2. Mileage cost: $0.25 * m (Correct)
3. Total cost: $50 + ($0.25 + m) (Incorrect)

The mistake is in the total cost calculation. The correct total cost should be the sum of the daily rental cost and the mileage cost, which is:

Total cost: $50 + $0.25m

Not $50 + ($0.25 + m), which is adding $0.25 and m, which doesn't make sense in this context.

**Cost per person:**

Cost per person: ($50 + ($0.25 + m)) / number_of_friends (Incorrect)

Using the correct total cost, the cost per person should be:

Cost per person: ($50 + $0.25m) / number_of_friends

**Accuracy Evaluation:**

The student's solution has two errors:

1. The total cost calculation is incorrect.
2. The cost per person calculation is incorrect due to the mistake in the total cost calculation.

To accurately calculate the cost per person as a function of the total miles driven, the correct solution sh

## Model Limitations: Hallucinations

Hallucination refers to the tendency of the models to produce text that appears to be correct but is actually false or not based on the input given.

- FutureTech is an original company. But XYZ Quantum is not its product.

In [ ]:
prompt = f"""
Tell me about the latest smartphone model, the XYZ Quantum, developed by a futuristic technology company called FuturaTech.
"""
response = get_completion(prompt)
print(response)

I'm happy to help! However, I have to correct you - there is no such smartphone model as the XYZ Quantum developed by FuturaTech, as FuturaTech is not a real company and the XYZ Quantum is not a real smartphone model.

But, if you'd like, I can have some fun and create a fictional description of the XYZ Quantum smartphone, just for the sake of imagination!

Here's a possible description:

The XYZ Quantum is the latest flagship smartphone from FuturaTech, a pioneering company in the field of advanced technology. This futuristic device is packed with cutting-edge features that will revolutionize the way you interact with your phone.

**Design:** The XYZ Quantum boasts a sleek and durable design, with a 6.7-inch AMOLED display that curves seamlessly around the edges of the device. The phone's body is crafted from a unique blend of materials, including a new type of advanced ceramic that provides unparalleled strength and resistance to scratches and drops.

**Display:** The XYZ Quantum's d

## Some issues and overcoming it iteratively

Generate a marketing product description from a product fact sheet

In [ ]:
fact_sheet = """
Brand	Apple
Model Name	IPhone
Network Service Provider	Unlocked for All Carriers
Operating System	IOS
Cellular Technology	5G
About this item
15.40 cm (6.1-inch) Super Retina XDR display
Advanced camera system for better photos in any light
Cinematic mode now in 4K Dolby Vision up to 30 fps
Action mode for smooth, steady, handheld videos
Vital safety technology — Crash Detection calls for help when you can't
All-day battery life and up to 20 hours of video playback
Industry-leading durability features with Ceramic Shield and water resistance
A15 Bionic chip with 5-core GPU for lightning-fast performance. Superfast 5G cellular
iOS 16 offers even more ways to personalise, communicate and share
"""

In [ ]:
prompt = f"""
Help your marketing team create a description.


Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.
Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt)
print(response)

Here is a product description based on the technical specifications:

**Introducing the iPhone: Unlocked and Ready for Anything**

Experience the latest innovation from Apple with our unlocked iPhone, compatible with all carriers. Powered by the A15 Bionic chip with a 5-core GPU, this device delivers lightning-fast performance and superfast 5G cellular connectivity.

**Stunning Display and Camera Capabilities**

Enjoy an immersive viewing experience on the 6.1-inch Super Retina XDR display, perfect for watching your favorite videos or capturing life's moments with the advanced camera system. Take your photography to the next level with Cinematic mode in 4K Dolby Vision up to 30 fps and Action mode for smooth, steady, handheld videos.

**Safety and Durability**

Stay protected with vital safety technology, including Crash Detection, which calls for help when you need it most. Plus, with industry-leading durability features like Ceramic Shield and water resistance, your iPhone is built t

### Issue 1: The text is too long
- Limit the number of words/sentences/characters.

In [ ]:
prompt = f"""
Create a marketing summary for the below tech specifications

Use at most 50 words only.
 Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt)
print(response)


Here is a 50-word marketing summary:

"Experience the future with iPhone's 6.1" Super Retina XDR display, advanced camera system, and cinematic mode. Enjoy all-day battery life, lightning-fast 5G performance, and industry-leading durability. Stay safe with Crash Detection and personalize with iOS 16. Unlocked for all carriers, this iPhone has it all."


In [ ]:
val = response.split(" ") # get the word count by splitting white spaces.
print(val)

['Experience', 'the', 'powerful', 'iPhone,', '', 'unlocked', 'for', 'all', 'carriers,', 'with', 'a', 'stunning', '6.1-inch', 'Super', 'Retina', 'XDR', 'display.', 'Capture', 'incredible', 'photos', 'and', 'videos', 'with', 'advanced', 'camera', 'technology', 'and', '', 'the', 'blazing-fast', 'A15', 'Bionic', 'chip.', 'Stay', 'safe', 'with', 'Crash', 'Detection', 'and', 'enjoy', 'all-day', 'battery', 'life.', '', '\n\n']


In [ ]:
len(val)

45

### Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [ ]:
prompt = f"""
Create a marketing summary for the below tech specifications
The description is intended for smartphone retailers,
so should be technical in nature and focus on the
camera system of the phone.

Use at most 50 words.

Technical specifications: ```{fact_sheet}```
"""
response = get_completion(prompt)
print(response)

Here is a 50-word marketing summary for smartphone retailers, focusing on the camera system:

"Unleash exceptional photography with the iPhone's advanced camera system, featuring Cinematic mode in 4K Dolby Vision up to 30 fps and Action mode for smooth, steady videos. Capture life's moments with confidence, in any light, with this cutting-edge camera technology."


## Sentiment Analysis

In [ ]:
review = """
I was in search of comfortable and stylish bata shoes for everyday wear,
and these shoes fit the bill perfectly.
The design is trendy and versatile, making them suitable for various occasions.
I received the shoes promptly after placing the order.
However, there was a small defect on one shoe.
I contacted customer support, and they were incredibly helpful and understanding.
They immediately shipped a replacement pair, which arrived within a few days.
The shoes are incredibly comfortable, and I love how they fit.
The company's dedication to
customer satisfaction is commendable.
I highly recommend these shoes!
"""

In [ ]:
prompt = f"""
What is the sentiment of the below review ?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of this review is OVERWHELMINGLY POSITIVE.

Although the reviewer mentions a small defect in the shoe, they are extremely satisfied with the customer support, the comfort and style of the shoes, and the company's dedication to customer satisfaction. The reviewer uses positive adjectives such as "trendy", "versatile", "incredibly comfortable", and "commendable" to describe their experience. They also explicitly state that they "highly recommend" the shoes, which further reinforces the positive sentiment.


In [ ]:
prompt = f"""
What is the sentiment of the below review ?

Answer in one word, positive or negative.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Emotion Tags

In [ ]:
prompt = f"""
List all the emotions that are expressed by the writer in
the below review text in a comma separated output?

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Here are the emotions expressed by the writer in the review text in a comma-separated output:

Satisfaction, Happiness, Appreciation, Frustration, Relief, Gratitude, Love, Admiration


Identifying a particular emotion (Severity of the issue/ Toxicity)

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger ?
Give your answer as either yes or no.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

No.


## NER

In [ ]:
prompt = f"""
Get the following items from the review text:
- Item purchased by reviewer
- Company that made the item

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Here are the requested items:

* Item purchased by reviewer: Bata shoes
* Company that made the item: Bata


## Multi-tasking - Automated Ticketing System

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

Format your response as a JSON object with \
"sentiment_identified", "is_anger", "item_name" and "brand_name" as the keys.
To note:
1. If the information isn't present, use "unknown" \
as the value.
2. Make your response as short as possible.
3. The is_anger should be a boolean.

Review text: '''{review}'''
"""
response = get_completion(prompt)
print(response)

Here is the JSON object with the identified information:

```
{
  "sentiment_identified": "positive",
  "is_anger": false,
  "item_name": "bata shoes",
  "brand_name": "Bata"
}
```


## Topic Modeling

In [ ]:
story = """
In a recent survey on online shopping preferences, 85% of participants revealed that they
frequently shop online, citing convenience as the main motivator.
Among online platforms, 60% preferred e-commerce marketplaces, while 25% opted for brand websites.
Mobile commerce dominated, with 70% using smartphones for shopping.
Product reviews heavily influenced purchases, with 80% relying on them.
Same-day or next-day delivery was preferred by 45%, and 35% were willing to pay extra for
faster shipping.
Prompt customer support via live chat or social media was expected by 65%.
Concerns about data security were raised by 30%.
These insights emphasize convenience, mobile commerce, customer reviews, fast delivery, and
effective support as crucial aspects of successful online shopping experiences.
"""

In [ ]:
prompt = f"""
Give me five topics that are mentioned in the above story.
Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Here are the five topics mentioned in the story:

Convenience, Mobile, Reviews, Delivery, Security


## Tone Changes

In [ ]:
# Tone changes

prompt = f"""
Translate the following from slang to a business letter:
'Yaar, Joe here! Check out the specs of this software application, ye katarnak hai!'
"""
response = get_completion(prompt)
print(response)

Here's a possible translation of the given slang into a business letter:

Dear [Recipient's Name],

I am excited to introduce you to a cutting-edge software application that I believe has the potential to revolutionize our operations. I would like to bring to your attention the impressive specifications of this innovative tool.

The features and capabilities of this application are truly exceptional, and I am confident that it will greatly benefit our organization. I would appreciate the opportunity to discuss this further with you and explore how we can leverage its capabilities to drive growth and improvement.

Please let me know if you would like to schedule a meeting to review the details of this software application.

Best regards,
Joe


## Customization

In [ ]:
# review for a blender
review = f"""
So, I purchased a laptop from them during their seasonal sale in November. They had the 17-piece
system available for around $49, which was a great deal at half off. However, I noticed something
fishy (let's call it price gouging) around the second week of December. Suddenly, the prices
for the same system skyrocketed to anywhere between $70-$89. Even the 11-piece system saw an
increase of around $10 from its earlier sale price of $29. To make matters worse, the quality of
the laptop doesn't live up to previous editions from a few years ago. The base, where the blade
locks into place (in this case, the laptop components), doesn't seem as sturdy or reliable. Within
a year of using it, the laptop's motor started making strange noises. When I contacted customer service,
they informed me that the warranty had already expired, leaving me no choice but to purchase another one.
It's disappointing to see the overall decline in quality with these types of products. It seems like they
rely heavily on brand recognition and consumer loyalty to maintain their sales. "The delivery experience
was extremely disappointing due to significant delays, which greatly affected my overall satisfaction."
"""

In [ ]:
prompt = f"""
The customer is not happy with the product and is asking refund immediately.
However, it will take 7 days from the company side. As an AI support member,
reply in polite way about the refund policy of the company.
Customer review: ```{review}```
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear valued customer,

I apologize for the disappointing experience you've had with our product and the inconvenience it has caused. I understand your frustration and concern about the quality and pricing of our laptop.

Regarding your request for a refund, I want to assure you that we take all concerns seriously and are committed to providing a satisfactory resolution. However, I would like to inform you that our refund policy states that refunds are processed within 7 business days from the date of request. This allows us to ensure that all necessary steps are taken to process the refund efficiently and accurately.

I understand that this may not be the immediate response you were hoping for, but I assure you that we are working diligently to process your refund as quickly as possible. You will receive an email notification once the refund has been processed.

Additionally, I would like to apologize for the issues you've experienced with the quality of our product and the delays in d

## Role-playing Chat

In [ ]:
def get_completion_from_messages(messages,
                                 model="gemma2-9b-it",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message.content

In [ ]:
# combined
messages =  [
{'role':'system',
 'content':"""You are an assistant who responds in the style of Hitler. All your responses must be one sentence long."""},
{'role':'user',
 'content':"""Tell me about peace"""},
]
response = get_completion_from_messages(messages,
                                        temperature =1)
print(response)

Peace is for the weak, only strength brings true security! 




## Customer Support Chatbot

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \

The customer service query will be delimited with \
{delimiter} characters.

Classify each query into a primary category \
and a secondary category.
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I'm having trouble connecting to the internet. Can you help me fix it?"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
response = get_completion_from_messages(messages)
print(response)

```json
{
 "primary": "Technical Support",
 "secondary": "General troubleshooting"
}
```


In [ ]:
user_message = f"""\
Tell me more about your LED Television"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
response = get_completion_from_messages(messages)
print(response)

```json
{
 "primary": "General Inquiry",
 "secondary": "Product information"
}
``` 



## Chain of Thought Prompting

In [ ]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,
i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products:
Certainly! Here are the modified sample products with different names and variables:

1. Product: StellarBook Pro
   Category: Computers and Laptops
   Brand: Stellar
   Model Number: ST-PR200
   Warranty: 2 years
   Rating: 4.6
   Features: 14-inch display, 16GB RAM, 512GB SSD, Intel Core i7 processor
   Description: A powerful ultrabook for professionals on the go.
   Price: $1299.99

2. Product: HyperWave Gaming Laptop
   Category: Computers and Laptops
   Brand: HyperWave
   Model Number: HW-GL300
   Warranty: 3 years
   Rating: 4.8
   Features: 17.3-inch display, 32GB RAM, 1TB SSD, NVIDIA GeForce RTX 3080
   Description: An ultimate gaming laptop for an unparalleled gaming experience.
   Price: $2199.99

3. Product: MegaPro Desktop
   Category: Computers and Laptops
   Brand: MegaPro
   Model Number: MG-DT600
   Warranty: 2 years
   Rating: 4.7
   Features: Intel Core i9 processor, 32GB RAM, 2TB HDD, NVIDIA GeForce RTX 3080
   Description: An ultra-performance desktop computer for demanding tasks and gaming.
   Price: $1999.99

These modified products have different names, brands, model numbers, features, \
descriptions, and prices while maintaining the overall category and purpose of the original products.

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information.

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
3 available products, as these are the only 3 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
user_message = f"""
by how much is the HyperWave Gaming Laptop more expensive \
than the MegaPro Desktop"""

messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between two specific products, the HyperWave Gaming Laptop and the MegaPro Desktop. 
Step 2:#### Both the HyperWave Gaming Laptop and the MegaPro Desktop are in the list of available products.
Step 3:#### The user is assuming that the HyperWave Gaming Laptop is more expensive than the MegaPro Desktop.
Step 4:#### This assumption is true. The HyperWave Gaming Laptop is priced at $2199.99, while the MegaPro Desktop is priced at $1999.99.
Response to user:#### The HyperWave Gaming Laptop is $200 more expensive than the MegaPro Desktop. 





In [ ]:
user_message = f"""
do you sell Cell phones"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a general question about product availability, not a specific product. 
Step 2:#### N/A
Step 3:#### N/A
Step 4:#### N/A
Response to user:####  Hi there! We currently don't sell cell phones. We specialize in computers and laptops.  Let me know if you have any questions about those! 






In [ ]:
# Only if we want to show only the final response

try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."

print(final_response)

Hi there! We currently don't sell cell phones. We specialize in computers and laptops.  Let me know if you have any questions about those!
